# 영어 Word2Vec 실습

### 파이썬의 genism 패키지의 Word2Vec 사용하기

**1) 파이썬의 genism 패키지에는 Word2Vec을 지원하고 있어 손쉽게 단어를 임베딩 벡터로 변환할 수 있음**

In [ ]:
import gensim
gensim.__version__

'3.6.0'

In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.0 MB/s 
     |████████████████████████████████| 465 kB 49.3 MB/s 


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

### 훈련 데이터 이해하기

**1) 훈련 데이터를 다운로드**

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7effe0d4a810>)

### 훈련 데이터 전처리하기

**1) xml 문법으로 작성된 파일에 대해 xml 문법들과 배경음을 나타내는 단어들을 제거하고 자연어 데이터만 가져오도록 전처리**<br>
**2) 또한 입력 코퍼스에 대해서 단어 토큰화를 수행**

In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

**3) 총 샘플의 개수는 27만 3천개 이며 상위 3개 문장을 출력하면 토큰화가 수행되었음을 볼 수 있음**

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [ ]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### Word2Vec 훈련시키기

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

**1) Word2Vec에 대해서 학습을 진행**

In [ ]:
# size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
# window = 컨텍스트 윈도우 크기
# min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
# workers = 학습을 위한 프로세스 수
# sg = 0은 CBOW, 1은 Skip-gram.
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

**2) Word2Vec은 입력한 단어에 대해서 가장 유사한 단어들을 출력하는 model.wv.most_similer을 지원하므로**<br>
&nbsp;&nbsp;&nbsp;&nbsp;     **Word2Vec을 통해 단어의 유사도를 계산하여 man과 가장 유사한 단어들을 출력해보도록 함**

In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8424811363220215), ('guy', 0.8290940523147583), ('boy', 0.7704038619995117), ('lady', 0.7589364051818848), ('girl', 0.7309212684631348), ('soldier', 0.7116029858589172), ('gentleman', 0.7100744247436523), ('kid', 0.6928150057792664), ('poet', 0.6735016107559204), ('friend', 0.6649017333984375)]


### Word2Vec 모델 저장하고 로드하기

**1) 학습한 모델을 언제든 다시 사용할 수 있도록 컴퓨터 파일에 저장하고 다시 로드해보도록 함**

In [ ]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

**2) 로드한 모델에 대해서 다시 man과 유사한 단어를 출력해보도록 함**

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8424811363220215), ('guy', 0.8290940523147583), ('boy', 0.7704038619995117), ('lady', 0.7589364051818848), ('girl', 0.7309212684631348), ('soldier', 0.7116029858589172), ('gentleman', 0.7100744247436523), ('kid', 0.6928150057792664), ('poet', 0.6735016107559204), ('friend', 0.6649017333984375)]
